In [0]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import pickle
import dill

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = "/content/gdrive/My Drive/vmware/Project/"

In [0]:
data = pd.read_csv("/content/gdrive/My Drive/vmware/Project/214551_finalised_dataset.csv", sep=",")

In [0]:
data = data.drop(columns=data.columns[0:2])

In [0]:
data

,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,wait_time,arrival,arrival_delay,delay_last,wait_time_last,arrival_-24,delay_-24,wait_time_-24,arrival_-48,delay_-48,wait_time_-48,arrival_-week,wait_time_-week,delay_-week
0,214551,06:29:21,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,17,06:46:00,4,17.0,17.0,06:46:00,17,17,06:46:00,17,17,06:46:00,17,17
1,214551,06:29:21,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,37,07:06:00,4,17.0,37.0,07:06:00,37,37,07:06:00,37,37,07:06:00,37,37
2,214551,06:29:21,07:26:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:22:00,4,0,84,0.0,9.6,0.7,57,07:26:00,4,37.0,57.0,07:26:00,57,57,07:26:00,57,57,07:26:00,57,57
3,214551,06:32:25,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,14,06:46:00,4,17.0,14.0,06:46:00,17,14,06:46:00,17,14,06:46:00,14,17
4,214551,06:32:25,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,34,07:06:00,4,17.0,34.0,07:06:00,37,34,07:06:00,37,34,07:06:00,34,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373014,214551,23:21:26,23:33:33,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,13,23:34:00,10,57.0,13.0,23:27:02,47,6,23:27:14,55,6,23:34:00,13,13
373015,214551,23:24:29,23:34:32,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10,23:34:00,10,57.0,10.0,23:27:02,47,3,23:27:14,55,3,23:34:00,10,10
373016,214551,23:27:32,23:37:21,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,7,23:34:00,10,57.0,7.0,23:27:02,47,0,23:27:14,55,0,23:34:00,7,7
373017,214551,23:30:34,23:34:52,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,4,23:34:00,10,57.0,4.0,23:27:02,47,-3,23:27:14,55,-3,23:34:00,4,4


In [0]:
data.query_time = pd.to_datetime(data.query_time)
data.query_time = [d.time() for d in data.query_time]
data["query_time_int"] = [(d.hour*60 + d.minute) for d in data.query_time]

In [0]:
data.expected_arrival = pd.to_datetime(data.expected_arrival)
data.expected_arrival = [d.time() for d in data.expected_arrival]
data["expected_arrival_int"] = [(d.hour*60 + d.minute) for d in data.expected_arrival]

In [0]:
data.timetable = pd.to_datetime(data.timetable)
data.timetable = [d.time() for d in data.timetable]
data["timetable_int"] = [(d.hour*60 + d.minute) for d in data.timetable]

In [0]:
data.arrival = pd.to_datetime(data.arrival)
data.arrival = [d.time() for d in data.arrival]
data["arrival_int"] = [(d.hour*60 + d.minute) for d in data.arrival]

In [0]:
data["arrival_-48"] = pd.to_datetime(data["arrival_-48"])
data["arrival_-48"] = [d.time() for d in data["arrival_-48"]]
data["arrival_-48_int"] = [(d.hour*60 + d.minute) for d in data["arrival_-48"]]

In [0]:
data["arrival_-week"] = pd.to_datetime(data["arrival_-week"])
data["arrival_-week"] = [d.time() for d in data["arrival_-week"]]
data["arrival_-week_int"] = [(d.hour*60 + d.minute) for d in data["arrival_-week"]]

In [0]:
data["arrival_-24"] = pd.to_datetime(data["arrival_-24"])
data["arrival_-24"] = [d.time() for d in data["arrival_-24"]]
data["arrival_-24_int"] = [(d.hour*60 + d.minute) for d in data["arrival_-24"]]

In [0]:
wait_time = data.drop(columns=["stop_no", "route_no", "query_time", "stop_name", "destination", "gps_tracked",
                               "hour", "wait_time", "wait_time_last", "wait_time_-24",
                               "wait_time_-48", "wait_time_-week", "expected_arrival",
                               "timetable", "arrival", "arrival_-24", "arrival_-48", "arrival_-week"])

In [0]:
wait_time

,date,day,is_bank_holiday,humidity,rain,temp,sun_hours,arrival_delay,delay_last,delay_-24,delay_-48,delay_-week,query_time_int,expected_arrival_int,timetable_int,arrival_int,arrival_-week_int,arrival_-24_int,arrival_-48_int
0,2016-06-17,4,0,84,0.0,9.6,0.7,4,17.0,17,17,17,389,406,402,406,406,406,406
1,2016-06-17,4,0,84,0.0,9.6,0.7,4,17.0,37,37,37,389,426,422,426,426,426,426
2,2016-06-17,4,0,84,0.0,9.6,0.7,4,37.0,57,57,57,389,446,442,446,446,446,446
3,2016-06-17,4,0,84,0.0,9.6,0.7,4,17.0,17,17,17,392,406,402,406,406,406,406
4,2016-06-17,4,0,84,0.0,9.6,0.7,4,17.0,37,37,37,392,426,422,426,426,426,426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373014,2017-09-24,6,0,93,0.0,11.3,0.0,10,57.0,47,55,13,1401,1413,1404,1414,1414,1407,1407
373015,2017-09-24,6,0,93,0.0,11.3,0.0,10,57.0,47,55,10,1404,1414,1404,1414,1414,1407,1407
373016,2017-09-24,6,0,93,0.0,11.3,0.0,10,57.0,47,55,7,1407,1417,1404,1414,1414,1407,1407
373017,2017-09-24,6,0,93,0.0,11.3,0.0,10,57.0,47,55,4,1410,1414,1404,1414,1414,1407,1407


In [0]:
wait_time = wait_time.drop_duplicates(["date", "timetable_int"])

In [0]:
wait_time

,date,day,is_bank_holiday,humidity,rain,temp,sun_hours,arrival_delay,delay_last,delay_-24,delay_-48,delay_-week,query_time_int,expected_arrival_int,timetable_int,arrival_int,arrival_-week_int,arrival_-24_int,arrival_-48_int
0,2016-06-17,4,0,84,0.0,9.6,0.7,4,17.0,17,17,17,389,406,402,406,406,406,406
1,2016-06-17,4,0,84,0.0,9.6,0.7,4,17.0,37,37,37,389,426,422,426,426,426,426
2,2016-06-17,4,0,84,0.0,9.6,0.7,4,37.0,57,57,57,389,446,442,446,446,446,446
12,2016-06-17,4,0,84,0.0,9.6,0.7,-2,57.0,58,58,58,398,456,458,456,456,456,456
39,2016-06-17,4,0,76,0.0,10.9,1.0,4,58.0,58,58,58,423,481,477,481,481,481,481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372904,2017-09-24,6,0,87,0.0,12.2,0.0,10,57.0,59,59,58,1276,1334,1324,1334,1334,1334,1334
372924,2017-09-24,6,0,87,0.0,12.2,0.0,10,58.0,57,56,58,1297,1354,1344,1354,1354,1354,1347
372942,2017-09-24,6,0,87,0.0,12.2,0.0,10,57.0,58,59,57,1316,1374,1364,1374,1374,1374,1374
372963,2017-09-24,6,0,89,0.0,11.9,0.0,10,58.0,58,58,57,1337,1394,1384,1394,1394,1394,1394


In [0]:
lin = LinearRegression()

In [0]:
X = wait_time.drop(columns=["date","arrival_int"])
y = wait_time.arrival_int

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = float(1/7), random_state=140363)

In [0]:
lin_pipeline = Pipeline([("estimator", LinearRegression())])

In [0]:
lin_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('estimator',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [0]:
lin_pipeline.predict(X_train)

array([ 951.,  931., 1009., ...,  711.,  621.,  691.])

In [0]:
y_train

47519      951
12876      931
351260    1009
150005     481
138852    1131
          ... 
160474     595
320051     831
340874     711
221237     621
81028      691
Name: arrival_int, Length: 18979, dtype: int64

In [0]:
np.mean(cross_val_score(lin_pipeline, X, y, scoring="neg_mean_absolute_error", cv=5))

-1.1177148130564798

In [0]:
rf_pipeline = Pipeline([("estimator", RandomForestRegressor())])

In [0]:
rf_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('estimator',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse', max_depth=None,
                                       max_features='auto', max_leaf_nodes=None,
                                       max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=100, n_jobs=None,
                                       oob_score=False, random_state=None,
                                       verbose=0, warm_start=False))],
         verbose=False)

In [0]:


rf_pipeline.predict(X_test)

array([1115.  ,  841.  ,  595.  , ...,  535.  , 1290.04,  426.  ])

In [0]:
skf = StratifiedKFold(n_splits=10)

np.mean(cross_val_score(rf_pipeline, X, y, scoring="neg_mean_absolute_error", cv=skf))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


-0.2056858362849982

execution time: 70.52s

In [0]:
rf_pipeline2 =  Pipeline([("estimator", RandomForestRegressor(n_estimators=200, random_state=140363))])

In [0]:
rf_pipeline3 =  Pipeline([("estimator", RandomForestRegressor(n_estimators=500, min_samples_split=4, min_samples_leaf=4, random_state=140363))])

In [0]:
rf_pipeline4 =  Pipeline([("estimator", RandomForestRegressor(n_estimators=1000, random_state=140363, n_jobs=5))])

In [0]:
rf_pipeline5 = Pipeline([("estimator", RandomForestRegressor(n_jobs=2, min_samples_leaf=2, min_samples_split=4))])

In [0]:
np.mean(cross_val_score(rf_pipeline2, X, y, scoring="neg_mean_absolute_error", cv=skf))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


-0.19299623226706206

execution time: 70.44 s

In [0]:
np.mean(cross_val_score(rf_pipeline3, X, y, scoring="neg_mean_absolute_error", cv=skf))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


-0.30143817353505514

execution time: 417.36s

In [0]:
np.mean(cross_val_score(rf_pipeline4, X, y, scoring="neg_mean_absolute_error", cv=skf))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


-0.2020207360099174

execution time: 953.74s

In [0]:
np.mean(cross_val_score(rf_pipeline5, X, y, scoring="neg_mean_absolute_error", cv=skf))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


-0.22944897253232374

execution time: 1006.45s

In [0]:
# Number of trees in random forest
n_estimators = [i for i in range(100, 1001, 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [i for i in range(10, 101, 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 4, 5, 7, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [0]:
rf = RandomForestRegressor(n_estimators=200, random_state=140363)

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  6.7min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 35.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 74.5min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=200,
                              

In [0]:
import dill

dill.dump_session("/content/gdrive/My Drive/vmware/Project/arrival_time.pkl")

In [0]:
gbr_pipeline = Pipeline([("estimator", GradientBoostingRegressor())])

In [0]:
gbr_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('estimator',
                 GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                           criterion='friedman_mse', init=None,
                                           learning_rate=0.1, loss='ls',
                                           max_depth=3, max_features=None,
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators=100,
                                           n_iter_no_change=None,
                                           presort='deprecated',
                                           random_state=N

In [0]:
gbr_pipeline.predict(X_test)

array([1114.95497534,  841.31152187,  595.07420023, ...,  535.44809719,
       1289.27461807,  426.09313939])

In [0]:
y_test

88435     1115
315275     841
363476     595
291525     836
30905      935
          ... 
336964    1095
220429    1001
364630     535
15017     1290
151493     426
Name: arrival_int, Length: 3164, dtype: int64

In [0]:
np.mean(cross_val_score(gbr_pipeline, X, y, scoring="neg_mean_absolute_error", cv = skf))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


-1.230659780493151

42.36s

In [0]:
gbr_pipeline2 = Pipeline([("estimator", GradientBoostingRegressor(learning_rate=.05, n_estimators=200))])

In [0]:
gbr_pipeline3 = Pipeline([("estimator", GradientBoostingRegressor(learning_rate=.2, n_estimators=300, min_samples_split=4))])

In [0]:
gbr_pipeline4 = Pipeline([("estimator", GradientBoostingRegressor(n_estimators=1000, max_depth = 20, min_samples_leaf=2))])

In [0]:
gbr_pipeline5 = Pipeline([("estimator", GradientBoostingRegressor(learning_rate=.075, n_estimators=750, max_depth = 20, 
                                                                  min_samples_split=4, min_samples_leaf=4))])

In [0]:
np.mean(cross_val_score(gbr_pipeline2, X, y, scoring="neg_mean_absolute_error"))

-0.5538539094296497

execution time: 37.26s

In [0]:
np.mean(cross_val_score(gbr_pipeline3, X, y, scoring="neg_mean_absolute_error"))

-0.5992157879058206

execution time: 56.63s

In [0]:
np.mean(cross_val_score(gbr_pipeline4, X, y, scoring="neg_mean_absolute_error"))

-0.23101243274917116

534.02s

In [0]:
np.mean(cross_val_score(gbr_pipeline5, X, y, scoring="neg_mean_absolute_error"))

-0.3421552670108531

1409.54s

In [0]:
import dill
dill.load_session("/content/drive/My Drive/vmware/Project/arrival_time.pkl")

In [0]:
gbr_base = GradientBoostingRegressor(n_estimators=1000, max_depth = 20, min_samples_leaf=2)

In [0]:
GradientBoostingRegressor()

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
learning_rate = [0.01, 0.05, 0.1, 0.2, 0.25, 0.5, 1.0]
n_estimators = [i for i in range(100, 1001, 100)]
max_depth = [i for i in range(3, 31, 3)]
max_features = ["auto", "sqrt", "log2"]
min_samples_leaf = [1,2,4]
min_samples_split = [2,4,8]

In [0]:
gbr_grid = {"learning_rate": learning_rate,
            "n_estimators": n_estimators,
            "max_depth": max_depth,
            "max_features": max_features,
            "min_samples_leaf": min_samples_leaf,
            "min_samples_split": min_samples_split}

In [0]:
gbr_random = RandomizedSearchCV(gdr_base, gdr_grid, 100, cv=3)

In [0]:
gbr_random.fit(X_train, y_train)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=20,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=2,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                            

In [0]:
dill.dump_session("/content/drive/My Drive/vmware/Project/arrival_time.pkl")

In [19]:
gbr_random

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=20,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=2,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                            

In [20]:
gbr_random.best_estimator_

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.2, loss='ls', max_depth=18,
                          max_features='auto', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [22]:
np.mean(cross_val_score(gbr_random.best_estimator_, X, y, scoring="neg_mean_absolute_error", cv=skf))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


-0.22287408977706374

In [23]:
rf_random.best_estimator_

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=100, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=3, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=140363, verbose=0, warm_start=False)

In [0]:
rf_best = rf_random.best_estimator_

In [25]:
np.mean(cross_val_score(rf_best, X, y, scoring="neg_mean_absolute_error", cv=skf))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


-0.22199026184693751